In [1]:
#import dependencies
import pandas as pd
import psycopg2
import sqlalchemy
import numpy as np
import matplotlib as plt
from scipy.stats import zscore, stats
import weightedstats as ws
import math

In [2]:
#create connection to postgres database
from sqlalchemy import create_engine

POSTGRES_ADDRESS = 'localhost' 
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' 
POSTGRES_PASSWORD = 'Woot123!'  
POSTGRES_DBNAME = 'Lahman DB' 

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))

engine = create_engine(postgres_str)


In [3]:
data = pd.read_sql_query('''SELECT * FROM batting;''', engine)
data.head()

,playerid,yearid,stint,teamid,lgid,g,ab,r,h,doubles,...,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
0,abercda01,1871,1,TRO,NA,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NA,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NA,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NA,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NA,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0
5,armstbo01,1871,1,FW1,NA,12,49,9,11,2,...,5.0,0.0,1.0,0,1.0,NaN,NaN,NaN,NaN,0.0
6,barkeal01,1871,1,RC1,NA,1,4,0,1,0,...,2.0,0.0,0.0,1,0.0,NaN,NaN,NaN,NaN,0.0
7,barnero01,1871,1,BS1,NA,31,157,66,63,10,...,34.0,11.0,6.0,13,1.0,NaN,NaN,NaN,NaN,1.0
8,barrebi01,1871,1,FW1,NA,1,5,1,1,1,...,1.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
9,barrofr01,1871,1,BS1,NA,18,86,13,13,2,...,11.0,1.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0


In [4]:
#replace NaN with 0
data.fillna(value=0, inplace=True)
data.head()

,playerid,yearid,stint,teamid,lgid,g,ab,r,h,doubles,...,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
0,abercda01,1871,1,TRO,NA,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,addybo01,1871,1,RC1,NA,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,0.0,0.0,0.0,0.0,0.0
2,allisar01,1871,1,CL1,NA,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,0.0,0.0,0.0,0.0,1.0
3,allisdo01,1871,1,WS3,NA,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,0.0,0.0,0.0,0.0,0.0
4,ansonca01,1871,1,RC1,NA,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,0.0,0.0,0.0,0.0,0.0
5,armstbo01,1871,1,FW1,NA,12,49,9,11,2,...,5.0,0.0,1.0,0,1.0,0.0,0.0,0.0,0.0,0.0
6,barkeal01,1871,1,RC1,NA,1,4,0,1,0,...,2.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0
7,barnero01,1871,1,BS1,NA,31,157,66,63,10,...,34.0,11.0,6.0,13,1.0,0.0,0.0,0.0,0.0,1.0
8,barrebi01,1871,1,FW1,NA,1,5,1,1,1,...,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
9,barrofr01,1871,1,BS1,NA,18,86,13,13,2,...,11.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
#calculate raw avg, obp, slg, ops
data ["avg"] = data ["h"] / data ["ab"]
data ["obp"] = (data ["h"] + data ["bb"] + data ["hbp"]) / (data ["ab"] + data ["bb"] + data ["hbp"] + data ["sh"] + data ["sf"])
data ["slg"] = ((data ["h"] - data ["doubles"] - data ["triples"] - data ["hr"]) + (2 * data ["doubles"]) + (3 * data ["triples"]) + (4 * data ["hr"])) / data ["ab"]
data ["ops"] = data ["slg"] + data ["obp"]
data.head()

,playerid,yearid,stint,teamid,lgid,g,ab,r,h,doubles,...,so,ibb,hbp,sh,sf,gidp,avg,obp,slg,ops
0,abercda01,1871,1,TRO,NA,1,4,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1,addybo01,1871,1,RC1,NA,25,118,30,32,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.271186,0.295082,0.322034,0.617116
2,allisar01,1871,1,CL1,NA,29,137,28,40,4,...,5.0,0.0,0.0,0.0,0.0,1.0,0.291971,0.302158,0.394161,0.696319
3,allisdo01,1871,1,WS3,NA,27,133,28,44,10,...,2.0,0.0,0.0,0.0,0.0,0.0,0.330827,0.330827,0.481203,0.812030
4,ansonca01,1871,1,RC1,NA,25,120,29,39,11,...,1.0,0.0,0.0,0.0,0.0,0.0,0.325000,0.336066,0.466667,0.802732
5,armstbo01,1871,1,FW1,NA,12,49,9,11,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.224490,0.224490,0.306122,0.530612
6,barkeal01,1871,1,RC1,NA,1,4,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.400000,0.250000,0.650000
7,barnero01,1871,1,BS1,NA,31,157,66,63,10,...,1.0,0.0,0.0,0.0,0.0,1.0,0.401274,0.447059,0.579618,1.026677
8,barrebi01,1871,1,FW1,NA,1,5,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.200000,0.400000,0.600000
9,barrofr01,1871,1,BS1,NA,18,86,13,13,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.151163,0.151163,0.197674,0.348837


In [6]:
#replace NaN with 0
data = data.replace(np.nan,0)
data.head()

,playerid,yearid,stint,teamid,lgid,g,ab,r,h,doubles,...,so,ibb,hbp,sh,sf,gidp,avg,obp,slg,ops
0,abercda01,1871,1,TRO,NA,1,4,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1,addybo01,1871,1,RC1,NA,25,118,30,32,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.271186,0.295082,0.322034,0.617116
2,allisar01,1871,1,CL1,NA,29,137,28,40,4,...,5.0,0.0,0.0,0.0,0.0,1.0,0.291971,0.302158,0.394161,0.696319
3,allisdo01,1871,1,WS3,NA,27,133,28,44,10,...,2.0,0.0,0.0,0.0,0.0,0.0,0.330827,0.330827,0.481203,0.812030
4,ansonca01,1871,1,RC1,NA,25,120,29,39,11,...,1.0,0.0,0.0,0.0,0.0,0.0,0.325000,0.336066,0.466667,0.802732
5,armstbo01,1871,1,FW1,NA,12,49,9,11,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.224490,0.224490,0.306122,0.530612
6,barkeal01,1871,1,RC1,NA,1,4,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.400000,0.250000,0.650000
7,barnero01,1871,1,BS1,NA,31,157,66,63,10,...,1.0,0.0,0.0,0.0,0.0,1.0,0.401274,0.447059,0.579618,1.026677
8,barrebi01,1871,1,FW1,NA,1,5,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.200000,0.400000,0.600000
9,barrofr01,1871,1,BS1,NA,18,86,13,13,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.151163,0.151163,0.197674,0.348837


In [84]:
#create function to calculate z-scores
def calculate (z):
    z = str(z)
#pick year and filter out players with less than 100 at bats
    year_x = data[data["yearid"]==z]  
    min_100_ab = year_x[year_x["ab"]>=100]

#calculate z-score for avg
    avg_avg = ws.numpy_weighted_mean(year_x["avg"], weights=year_x["ab"])
    min_100_ab["avg_zscore"] = (min_100_ab["avg"] - avg_avg)/min_100_ab["avg"].std(ddof=0)
      
#calculate z-score for obp
    avg_obp = ws.numpy_weighted_mean(year_x["obp"], weights=year_x["ab"])
    min_100_ab["obp_zscore"] = (min_100_ab["obp"] - avg_obp)/min_100_ab["obp"].std(ddof=0)
    
#calculate z-score for slg
    avg_slg = ws.numpy_weighted_mean(year_x["slg"], weights=year_x["ab"])
    min_100_ab["slg_zscore"] = (min_100_ab["slg"] - avg_slg)/min_100_ab["slg"].std(ddof=0)
    
#calculate z-score for ops
    avg_ops = ws.numpy_weighted_mean(year_x["ops"], weights=year_x["ab"])
    min_100_ab["ops_zscore"] = (min_100_ab["ops"] - avg_ops)/min_100_ab["ops"].std(ddof=0)
    
    return(min_100_ab)


total_dataset = pd.DataFrame()

for i in range (1871, 2019):
    temp = calculate (i)
    total_dataset = pd.concat([total_dataset, temp])
total_dataset



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://

,playerid,yearid,stint,teamid,lgid,g,ab,r,h,doubles,...,sf,gidp,avg,obp,slg,ops,avg_zscore,obp_zscore,slg_zscore,ops_zscore
1,addybo01,1871,1,RC1,NA,25,118,30,32,6,...,0.0,0.0,0.271186,0.295082,0.322034,0.617116,-0.271593,-0.283223,-0.607695,-0.506004
2,allisar01,1871,1,CL1,NA,29,137,28,40,4,...,0.0,1.0,0.291971,0.302158,0.394161,0.696319,0.095925,-0.157672,0.101386,0.009278
3,allisdo01,1871,1,WS3,NA,27,133,28,44,10,...,0.0,0.0,0.330827,0.330827,0.481203,0.812030,0.782997,0.350980,0.957105,0.762076
4,ansonca01,1871,1,RC1,NA,25,120,29,39,11,...,0.0,0.0,0.325000,0.336066,0.466667,0.802732,0.679960,0.443923,0.814198,0.701586
7,barnero01,1871,1,BS1,NA,31,157,66,63,10,...,0.0,1.0,0.401274,0.447059,0.579618,1.026677,2.028666,2.413206,1.924627,2.158532
15,bellast01,1871,1,TRO,NA,29,128,26,32,3,...,0.0,2.0,0.250000,0.299270,0.320312,0.619583,-0.646220,-0.208916,-0.624618,-0.489956
20,birdge01,1871,1,RC1,NA,25,106,19,28,2,...,0.0,3.0,0.264151,0.284404,0.377358,0.661762,-0.395997,-0.472681,-0.063796,-0.215542
21,birdsda01,1871,1,BS1,NA,29,152,51,46,3,...,0.0,1.0,0.302632,0.320513,0.361842,0.682355,0.284433,0.167981,-0.216338,-0.081569
22,brainas01,1871,1,WS3,NA,30,134,24,30,4,...,0.0,1.0,0.223881,0.262411,0.253731,0.516143,-1.108073,-0.862877,-1.279181,-1.162919
26,carleji01,1871,1,CL1,NA,29,127,31,32,8,...,0.0,3.0,0.251969,0.296296,0.330709,0.627005,-0.611412,-0.261678,-0.522413,-0.441667


In [85]:
mendoza = total_dataset[total_dataset["playerid"]=="mendoma01"]
mendoza

# mendoza_line = mendoza["avg_zscore"].mean()
# mendoza_line

,playerid,yearid,stint,teamid,lgid,g,ab,r,h,doubles,...,sf,gidp,avg,obp,slg,ops,avg_zscore,obp_zscore,slg_zscore,ops_zscore
52329,mendoma01,1974,1,PIT,NL,91,163,10,36,1,...,2.0,6.0,0.220859,0.254237,0.251534,0.505771,-1.118678,-1.591676,-1.712224,-1.780474
56978,mendoma01,1979,1,SEA,AL,148,373,26,74,10,...,5.0,12.0,0.198391,0.209476,0.249330,0.458806,-1.935900,-2.693997,-1.809865,-2.267017
57937,mendoma01,1980,1,SEA,AL,114,277,27,68,6,...,1.0,5.0,0.245487,0.275410,0.310469,0.585879,-0.559287,-1.114585,-1.039347,-1.151675
58878,mendoma01,1981,1,TEX,AL,88,229,18,53,6,...,3.0,9.0,0.231441,0.240157,0.266376,0.506533,-0.682308,-1.822741,-1.444733,-1.708550
